# Core

Core functionality for ShellSage CLI

In [ ]:
#|default_exp core

In [ ]:
#|export
from fastcore.script import *
from pathlib import Path
import os
import sys
import shell_sage.utils as utils

In [ ]:
#|export
def main(
    query: Param("The query to send to the LLM", str, nargs='*') = None,  # Make query optional
    pid: Param("Current, all or tmux pane_id for context", str) = 'current',
    skip_system: Param("Skip system information in AI context", store_true) = False,
    history_lines: Param("Number of history lines", int) = None,
    s: Param("Enable sassy mode", store_true) = False,
    c: Param("Enable command mode", store_true) = False,
    provider: Param("The LLM Provider", str) = None,
    model: Param("The LLM model", str) = None,
    base_url: Param("Base URL for API", str) = None,
    api_key: Param("API key", str) = None,
    code_theme: Param("Code theme for responses", str) = None,
    code_lexer: Param("Lexer for inline code", str) = None,
    index_man: Param("Index all man pages", store_true) = False,
    man_query: Param("Query indexed man pages", str) = None,
    db_path: Param("Path to LanceDB database", str) = "man_index.lance",
    verbosity: Param("Level of verbosity (0 or 1)", int) = 0
):
    """ShellSage CLI for interacting with LLMs and querying man pages."""
    # Handle RAG operations first
    if index_man:
        import shell_sage.rag as rag
        rag.index_cmd(db_path)
        return
        
    if man_query:
        import shell_sage.rag as rag
        rag.search_cmd(man_query, db_path=db_path)
        return

    # Require query for normal operation
    if not query:
        raise ValueError("Query is required unless using --index-man or --man-query")
    
    opts = utils.get_opts(history_lines=history_lines, provider=provider, model=model,
                   base_url=base_url, api_key=api_key, code_theme=code_theme,
                   code_lexer=code_lexer)

    mode = 'default'
    if s: mode = 'sassy'
    if c: mode = 'command'

    context = utils.get_context(pid=pid, skip_system=skip_system)
    response = utils.get_response(query, context, mode=mode, **opts)
    print(response)